In [1]:
import glob
import pandas as pd
import json
import os

find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL1"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL2"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL3"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL4"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL5"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL6"
# find_path = "../차량 내 탑승자 상황 인식 영상 데이터/데이터/Training/라벨링데이터/abnormal_230303_add/TL7"
file_list = glob.glob(os.path.join(find_path, "*/*/label/*.json"))

len(file_list)

74836

In [10]:
# 모든 데이터에 occupant는 한명
check = False

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())
    if len(js['occupant_info']) != 1:
        check = True
        break

print(check)

True


In [11]:
# 모든 이미지에 occupant는 한명
check = False

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())

    if len(js['occupant_info']) != 1:
        check = True
    
    scene = js['scene']['data']
    for img in scene:
        if len(img['occupant']) != 1:
            check = True
    if check:
        break

print(check)

True


# 데이터 변환

In [12]:
data = []

for file in file_list:
    with open(file) as f:
        js = json.loads(f.read())

    video_id = js['metadata']['video_id']
    date = js['metadata']['date']
    scene_id = js['scene_info']['scene_id']
    category_id = js['scene_info']['category_id']
    category_name = js['scene_info']['category_name']
    occupant_id = js['occupant_info'][0]['occupant_id']
    occupant_sex = js['occupant_info'][0]['occupant_sex']
    occupant_age = js['occupant_info'][0]['occupant_age']
    occupant_position = js['occupant_info'][0]['occupant_posotion'] # posotion이라고 잘못되어있음

    scene = js['scene']['data']
    for img in scene:
        data_length = len(scene)
        img_id = img['img_name']
        # occupant_id = img['occupant'][0]['occupant_id'] # 이미 있음
        b_bbox1, b_bbox2, b_bbox3, b_bbox4 = img['occupant'][0]['body_b_box']
        f_bbox1, f_bbox2, f_bbox3, f_bbox4 = img['occupant'][0]['face_b_box']
        action = img['occupant'][0]['action']
        emotion = img['occupant'][0]['emotion']

        data.append([video_id, date, scene_id, category_id, category_name, occupant_id, occupant_sex, occupant_age, occupant_position, data_length, img_id, b_bbox1, b_bbox2, b_bbox3, b_bbox4, f_bbox1, f_bbox2, f_bbox3, f_bbox4, action, emotion])

In [13]:
df = pd.DataFrame(data=data, columns=['video_id', 'date', 'scene_id', 'category_id', 'category_name', 'occupant_id', 'occupant_sex', 'occupant_age', 'occupant_position', 'data_length', 'img_id', 'b_bbox1', 'b_bbox2', 'b_bbox3', 'b_bbox4', 'f_bbox1', 'f_bbox2', 'f_bbox3', 'f_bbox4', 'action', 'emotion'])

In [14]:
df

,video_id,date,scene_id,category_id,category_name,occupant_id,occupant_sex,occupant_age,occupant_position,data_length,...,b_bbox1,b_bbox2,b_bbox3,b_bbox4,f_bbox1,f_bbox2,f_bbox3,f_bbox4,action,emotion
0,SGA2101132,20220104,SGA2101132S0003,A001,졸음운전,occupant1,M,30대,front,5,...,507.786415,110.745464,602.613585,601.254536,707.200000,117.145464,193.6,236.454536,하품,중립
1,SGA2101132,20220104,SGA2101132S0003,A001,졸음운전,occupant1,M,30대,front,5,...,507.786415,110.745464,605.813585,604.454536,707.200000,117.145464,196.8,239.654536,하품,중립
2,SGA2101132,20220104,SGA2101132S0003,A001,졸음운전,occupant1,M,30대,front,5,...,507.786415,110.745464,609.013585,607.654536,707.200000,117.145464,200.0,242.854536,하품,중립
3,SGA2101132,20220104,SGA2101132S0003,A001,졸음운전,occupant1,M,30대,front,5,...,509.386415,104.345464,612.213585,610.854536,633.600002,118.745464,203.2,246.054536,꾸벅꾸벅졸다,중립
4,SGA2101132,20220104,SGA2101132S0003,A001,졸음운전,occupant1,M,30대,front,5,...,509.386415,94.745463,615.413585,614.054536,673.600001,104.345464,206.4,249.254536,꾸벅꾸벅졸다,중립
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374172,SGA2101407,20220111,SGA2101407S0525,A007,운전자폭행,occupant1,F,30대,front,5,...,698.239990,164.800000,576.960010,555.200000,899.839990,171.235556,214.4,310.256072,운전하다,놀람
374173,SGA2101407,20220111,SGA2101407S0525,A007,운전자폭행,occupant1,F,30대,front,5,...,698.239990,164.800000,580.160010,558.400000,899.839990,171.235556,217.6,313.456072,운전하다,놀람
374174,SGA2101407,20220111,SGA2101407S0525,A007,운전자폭행,occupant1,F,30대,front,5,...,698.239990,164.800000,579.200000,555.200000,899.839990,171.235556,220.8,316.656072,운전하다,놀람
374175,SGA2101407,20220111,SGA2101407S0525,A007,운전자폭행,occupant1,F,30대,front,5,...,698.239990,164.800000,574.400000,546.886828,899.839990,171.235556,224.0,319.856072,운전하다,놀람


In [15]:
df.describe()

,data_length,b_bbox1,b_bbox2,b_bbox3,b_bbox4,f_bbox1,f_bbox2,f_bbox3,f_bbox4
count,374177.000000,374177.000000,374177.000000,374177.000000,374177.000000,374177.000000,374177.000000,374177.000000,374177.000000
mean,4.999968,645.006106,85.790188,605.369638,633.850784,843.793094,101.406883,247.096406,299.337144
std,0.005663,115.478943,55.984387,104.596253,58.710892,110.410228,56.353942,46.786095,55.425226
min,4.000000,0.000000,-1.600000,222.400000,235.580082,110.400000,-1.600000,40.000000,131.285117
25%,5.000000,567.039990,37.131249,544.960010,594.670933,771.200000,54.400000,208.000000,255.666183
50%,5.000000,642.421252,94.400000,601.600000,625.600000,843.018737,108.801580,245.411806,296.325000
75%,5.000000,704.000000,121.925000,658.560010,680.345490,913.600000,137.797406,279.039990,337.592606
max,5.000000,1008.000000,475.210092,1280.731314,812.800000,1240.000000,545.562686,830.400015,732.800000


In [3]:
df.to_csv('./validation_TL1.csv', index=False)
# df.to_csv('./validation_TL2.csv', index=False)
# df.to_csv('./validation_TL3.csv', index=False)
# df.to_csv('./validation_TL4.csv', index=False)
# df.to_csv('./validation_TL5.csv', index=False)
# df.to_csv('./validation_TL6.csv', index=False)
# df.to_csv('./validation_TL7.csv', index=False)

NameError: name 'df' is not defined